In [1]:
import os
import nltk
import logging
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
import google.generativeai as genai
from google.oauth2 import service_account
from pydantic import BaseModel, Field
from rouge_score import rouge_scorer
from collections import Counter
import re
import sacrebleu
from collections import Counter
from docx import Document as DocxDocument
from tqdm import tqdm
from langchain.docstore.document import Document as LangchainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
from bert_score import score
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [10]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [11]:
# Download required NLTK resources
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\farha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\farha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\farha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Data Preprocessing

In [12]:
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [13]:
# Define text preprocessing function with lemmatization
def preprocess_text(text):
    # 1. Strip whitespace
    text = text.strip()

    # 2. Remove special characters and numbers (optional, depending on requirements)
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # 3. Tokenize the text
    tokens = nltk.word_tokenize(text)

    # 4. Remove stopwords and apply lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(token.lower()) for token in tqdm(tokens, desc="Processing tokens") if token.lower() not in stop_words]

    # 5. Join the tokens back into a string
    preprocessed_text = " ".join(lemmatized_tokens)

    return preprocessed_text

In [14]:
# Read .docx files from 'dataset/word_standards' folder
def read_docx_files(folder_path):
    documents = []
    for filename in tqdm(os.listdir(folder_path), desc="Reading .docx files"):
        if filename.endswith(".docx"):
            file_path = os.path.join(folder_path, filename)
            try:
                docx_doc = DocxDocument(file_path)
                full_text = []
                for para in docx_doc.paragraphs:
                    full_text.append(para.text)
                text = "\n".join(full_text)
                # Create a LangChain Document object with text and metadata
                langchain_doc = LangchainDocument(page_content=text, metadata={"source": filename})
                documents.append(langchain_doc)
            except Exception as e:
                logger.error(f"Error reading {file_path}: {e}")
    return documents

In [15]:
# Load and preprocess documents from the folder
folder_path = "../data/word_standards"
documents = read_docx_files(folder_path)

Reading .docx files: 100%|██████████| 91/91 [00:04<00:00, 18.31it/s]


In [16]:
# Preprocess the text in each document
for doc in documents:
    doc.page_content = preprocess_text(doc.page_content)

Processing tokens: 100%|██████████| 15170/15170 [00:00<00:00, 459633.98it/s]


### Saving to Vector database

In [17]:
# Initialize the text splitter with overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=600,
    separators=["\n\n", "\n", " ", "", "\t", "\r\n", "\r", "\v", "\f", "\u0085", "\u2028", "\u2029"]
)

In [18]:
# Split the documents into chunks
split_docs = text_splitter.split_documents(documents)

logger.info(f"Total number of documents after splitting: {len(split_docs)}")

INFO:__main__:Total number of documents after splitting: 792


In [19]:
# Initialize the embeddings model
embedding_model_name = "all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Initialize FAISS vector store
vector_store = FAISS.from_documents(split_docs, embeddings)

# Save the vector store locally
vector_store.save_local("faiss_index")

# # To load the vector store from disk
# vector_store = FAISS.load_local("faiss_index", embeddings)

C:\Users\farha\AppData\Local\Temp\ipykernel_9580\2581478539.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
c:\Users\farha\miniconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.

### Gemini API configuration

In [20]:
# Path to the service account's JSON file
service_account_path = "adv-nlp-uts-faa7595a22eb.json"

# Create credentials using the service account JSON file
try:
    credentials = service_account.Credentials.from_service_account_file(service_account_path, scopes=["https://www.googleapis.com/auth/generative-language"])
except FileNotFoundError:
    logger.error(f"Service account file not found at {service_account_path}.")
    raise
except Exception as e:
    logger.error(f"Error creating credentials from the service account file: {e}")
    raise

# Configure the Gemini API client with the credentials
genai.configure(credentials=credentials)

### Configuring Gemini-1.5-Flash as serving LLM client

In [21]:
# Implement the Gemini LLM class
class GeminiLLM(LLM, BaseModel):
    model_name: str = Field(default="gemini-1.5-flash")
    temperature: float = Field(default=0.7)

    @property
    def _llm_type(self):
        return "gemini"

    def _call(self, prompt: str, stop: list[str] = None) -> str:
        try:
            # Initialize the model
            model = genai.GenerativeModel(model_name=self.model_name)

            # Generate content using the Gemini API
            response = model.generate_content(
                prompt,
                # temperature=self.temperature,
                # max_output_tokens=512  # Adjust token limit as needed
            )

            # Extract generated text from the response
            generated_text = response.text

            # Handle stop tokens if provided
            if stop:
                for token in stop:
                    generated_text = generated_text.split(token)[0]

            return generated_text.strip()

        except Exception as e:
            logger.error(f"Gemini API error: {e}")
            return "I'm sorry, but I couldn't process your request at this time."

In [22]:
# Initialize the Gemini LLM client
llm = GeminiLLM(model_name="gemini-1.5-flash", temperature=0.7)

# Define a prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an AI assistant with professional expertise in financial regulations and banking statistics, particularly knowledgeable about Australian APRA guidelines.
Based on the provided context, please answer the following question in a clear, well detailed, and informative manner. 
Ensure your response directly addresses the query and do not generate information that is not supported by the provided content.

Context:
{context}

Question:
{question}

Answer:
""",
)

### Configuring the RAG agent

In [23]:
# Create a RetrievalQA chain with the custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # You can experiment with 'refine' or 'map_reduce'
    retriever=vector_store.as_retriever(search_kwargs={"k": 100}),
    chain_type_kwargs={"prompt": prompt_template},
    return_source_documents=True,
)

In [24]:
# Function to handle user queries
def answer_query(query):
    try:
        response = qa_chain({"query": query})
        answer = response["result"]
        return answer
    except Exception as e:
        logger.error(f"Error during query processing: {e}")

In [25]:
# # Function to handle user queries
# def answer_query(query):
#     try:
#         response = qa_chain({"query": query})
#         answer = response["result"]
#         # source_docs = response["source_documents"]
#         # print("Response:")
#         # print(answer)
#         # print("\nRelevant Source Documents:")
#         # for doc in source_docs:
#         #     print(f"Source: {doc.metadata.get('source', 'Unknown Source')}")
#         #     print(doc.page_content)
#         #     print("-" * 80)
#         return answer
#     except Exception as e:
#         logger.error(f"Error during query processing: {e}")

##### Testing the RAG agent

In [26]:
# Test the RAG system with a query
query = "What is RWA?"
# query = "What is the quality control on International Banking Statistics Balance Sheet Items?"
# query = "What are the key elements in the definition of Effective Maturity?"
answer_query(query)

C:\Users\farha\AppData\Local\Temp\ipykernel_9580\3558265379.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": query})


'RWA stands for **risk-weighted asset**. It is the total value of an asset multiplied by the risk weight assigned to that asset class, as outlined in the prudential standard APS Capital Adequacy Standardised Approach Credit Risk.'

### Evaluation of RAG Agent

In [32]:
csv_path = "../data/question-answers.csv"
try:
    # Load the CSV file with a specified encoding
    data = pd.read_csv(csv_path, encoding="utf-8")
except UnicodeDecodeError:
    # If utf-8 fails, try a different encoding
    data = pd.read_csv(csv_path, encoding="ISO-8859-1")

In [33]:
# def evaluate_rouge(predicted, reference):
#     if predicted is None:
#         predicted = ""
#     if reference is None:
#         reference = ""
#     scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
#     return scorer.score(reference, predicted)


# def evaluate_bleu(predicted, reference):
#     if predicted is None:
#         predicted = ""
#     if reference is None:
#         reference = ""
#     bleu = sacrebleu.corpus_bleu([predicted], [[reference]])
#     return bleu.score


# def evaluate_f1(predicted, reference):
#     # Handling None values
#     if predicted is None:
#         predicted = ""
#     if reference is None:
#         reference = ""

#     # Preprocess both predicted and reference
#     predicted_tokens = nltk.word_tokenize(preprocess_text(predicted))
#     reference_tokens = nltk.word_tokenize(preprocess_text(reference))

#     # Use Counter to keep token frequency for precision and recall calculations
#     predicted_counts = Counter(predicted_tokens)
#     reference_counts = Counter(reference_tokens)

#     # Calculate true positives (common tokens)
#     common_tokens = sum((predicted_counts & reference_counts).values())

#     # Calculate precision and recall
#     precision = common_tokens / sum(predicted_counts.values()) if predicted_counts else 0
#     recall = common_tokens / sum(reference_counts.values()) if reference_counts else 0

#     # Calculate F1 score
#     if precision + recall == 0:
#         return 0.0
#     return 2 * (precision * recall) / (precision + recall)

In [34]:
def evaluate_rouge(predicted, reference):
    if predicted is None:
        predicted = ""
    if reference is None:
        reference = ""
    scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
    return scorer.score(reference, predicted)


def evaluate_bleu(predicted, reference):
    if predicted is None:
        predicted = ""
    if reference is None:
        reference = ""
    bleu = sacrebleu.corpus_bleu([predicted], [[reference]])
    return bleu.score


def evaluate_f1(predicted, reference):
    if predicted is None:
        predicted = ""
    if reference is None:
        reference = ""
    predicted_tokens = nltk.word_tokenize(preprocess_text(predicted))
    reference_tokens = nltk.word_tokenize(preprocess_text(reference))
    predicted_counts = Counter(predicted_tokens)
    reference_counts = Counter(reference_tokens)

    common_tokens = sum((predicted_counts & reference_counts).values())

    precision = common_tokens / sum(predicted_counts.values()) if predicted_counts else 0
    recall = common_tokens / sum(reference_counts.values()) if reference_counts else 0

    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)


def evaluate_bert_score(predicted, reference):
    if predicted is None:
        predicted = ""
    if reference is None:
        reference = ""
    P, R, F1 = score([predicted], [reference], lang="en", verbose=False)
    return F1.mean().item()


# Initialize SentenceTransformer for embedding similarity
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")


def evaluate_embedding_similarity(predicted, reference):
    if predicted is None:
        predicted = ""
    if reference is None:
        reference = ""
    predicted_embedding = sentence_model.encode(predicted)
    reference_embedding = sentence_model.encode(reference)
    similarity = cosine_similarity([predicted_embedding], [reference_embedding])[0][0]
    return similarity


def evaluate_hybrid(predicted, reference):
    rouge = evaluate_rouge(predicted, reference)
    rouge_score = rouge["rougeL"].fmeasure

    bleu_score = evaluate_bleu(predicted, reference)
    f1_score = evaluate_f1(predicted, reference)
    bert_score = evaluate_bert_score(predicted, reference)
    embedding_similarity = evaluate_embedding_similarity(predicted, reference)

    # Define a weighted hybrid score
    hybrid_score = (0.2 * rouge_score) + (0.2 * bleu_score) + (0.2 * f1_score) + (0.2 * bert_score) + (0.2 * embedding_similarity)
    return hybrid_score

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [35]:
# Add a new column for predicted answers
data["RAG 1"] = ""

# Initialize list to hold the predicted answers
predicted_answers = []

# Initialize lists to hold metric scores
rouge_scores, bleu_scores, f1_scores, bert_scores, embedding_similarities, hybrid_scores = [], [], [], [], [], []

# Initialize min and max values for each metric
rouge_min, rouge_max = float("inf"), float("-inf")
bleu_min, bleu_max = float("inf"), float("-inf")
f1_min, f1_max = float("inf"), float("-inf")
bert_min, bert_max = float("inf"), float("-inf")
embedding_min, embedding_max = float("inf"), float("-inf")

In [ ]:
# First pass to collect metric values and determine min and max for normalization
for index, row in tqdm(data.iterrows(), total=len(data), desc="Processing questions", leave=False):
    question = row["Question"]
    reference_answer = row["Answer"]
    predicted_answer = answer_query(question)
    predicted_answers.append(predicted_answer)

    # Evaluate the metrics
    rouge = evaluate_rouge(predicted_answer, reference_answer)
    bleu = evaluate_bleu(predicted_answer, reference_answer)
    f1 = evaluate_f1(predicted_answer, reference_answer)
    bert_score = evaluate_bert_score(predicted_answer, reference_answer)
    embedding_similarity = evaluate_embedding_similarity(predicted_answer, reference_answer)

    # Update min and max values for each metric
    rouge_scores.append(rouge["rougeL"].fmeasure)
    bleu_scores.append(bleu)
    f1_scores.append(f1)
    bert_scores.append(bert_score)
    embedding_similarities.append(embedding_similarity)

    rouge_min, rouge_max = min(rouge_min, rouge["rougeL"].fmeasure), max(rouge_max, rouge["rougeL"].fmeasure)
    bleu_min, bleu_max = min(bleu_min, bleu), max(bleu_max, bleu)
    f1_min, f1_max = min(f1_min, f1), max(f1_max, f1)
    bert_min, bert_max = min(bert_min, bert_score), max(bert_max, bert_score)
    embedding_min, embedding_max = min(embedding_min, embedding_similarity), max(embedding_max, embedding_similarity)

In [37]:
# Second pass to normalize the metrics and calculate the hybrid score
for index in tqdm(range(len(data)), desc="Calculating hybrid scores"):
    rouge = rouge_scores[index]
    bleu = bleu_scores[index]
    f1 = f1_scores[index]
    bert_score = bert_scores[index]
    embedding_similarity = embedding_similarities[index]

    # Normalize each metric
    rouge_normalized = (rouge - rouge_min) / (rouge_max - rouge_min) if rouge_max != rouge_min else 0.5
    bleu_normalized = (bleu - bleu_min) / (bleu_max - bleu_min) if bleu_max != bleu_min else 0.5
    f1_normalized = (f1 - f1_min) / (f1_max - f1_min) if f1_max != f1_min else 0.5
    bert_normalized = (bert_score - bert_min) / (bert_max - bert_min) if bert_max != bert_min else 0.5
    embedding_normalized = (embedding_similarity - embedding_min) / (embedding_max - embedding_min) if embedding_max != embedding_min else 0.5

    # Calculate hybrid score using normalized values
    hybrid_score = (0.2 * rouge_normalized) + (0.2 * bleu_normalized) + (0.2 * f1_normalized) + (0.2 * bert_normalized) + (0.2 * embedding_normalized)
    hybrid_scores.append(hybrid_score)

    # Save the predicted answer and hybrid score in the DataFrame
    predicted_answer = predicted_answers[index]
    data.at[index, "RAG 1"] = predicted_answer

    # Print each result
    print(f"[Processing question {index + 1}]:\n")
    print(f"ROUGE-L: {rouge}")
    print(f"BLEU: {bleu}")
    print(f"F1: {f1}")
    print(f"BERT Score: {bert_score}")
    print(f"Embedding Similarity: {embedding_similarity}")
    print(f"Hybrid Score: {hybrid_score}")
    print("-" * 80)

# Save the DataFrame with predictions and hybrid scores to a new CSV file
data.to_csv("../data/answers_rag1.csv", index=False)

Calculating hybrid scores: 100%|██████████| 80/80 [00:00<00:00, 20012.19it/s]

[Processing question 1]:

ROUGE-L: 0.14457831325301204
BLEU: 2.1562467214096475
F1: 0.19607843137254904
BERT Score: 0.8371809124946594
Embedding Similarity: 0.7557029128074646
Hybrid Score: 0.40106731357230574
--------------------------------------------------------------------------------
[Processing question 2]:

ROUGE-L: 0.23148148148148145
BLEU: 2.99386624213049
F1: 0.2201834862385321
BERT Score: 0.8727231621742249
Embedding Similarity: 0.770326554775238
Hybrid Score: 0.5546326071605248
--------------------------------------------------------------------------------
[Processing question 3]:

ROUGE-L: 0.1323529411764706
BLEU: 1.17170371353362
F1: 0.16279069767441862
BERT Score: 0.8237793445587158
Embedding Similarity: 0.5661188364028931
Hybrid Score: 0.29283189566480866
--------------------------------------------------------------------------------
[Processing question 4]:

ROUGE-L: 0.19718309859154928
BLEU: 0.8617830041262791
F1: 0.2
BERT Score: 0.8569048643112183
Embedding Simila

In [38]:
# Save the evaluation metrics to a CSV file
evaluation_metrics = {
    "ROUGE-L": rouge_scores,
    "BLEU": bleu_scores,
    "F1": f1_scores,
    "BERT Score": bert_scores,
    "Embedding Similarity": embedding_similarities,
    "Hybrid Score": hybrid_scores
}
evaluation_df = pd.DataFrame(evaluation_metrics)

evaluation_df.to_csv("../data/rag1_evaluation_metrics.csv", index=False)

# Print evaluation summary
print("=== Evaluation Summary ===")
print(f"Average ROUGE-L: {sum(rouge_scores) / len(rouge_scores):.4f}")
print(f"Average BLEU: {sum(bleu_scores) / len(bleu_scores):.2f}")
print(f"Average F1: {sum(f1_scores) / len(f1_scores):.4f}")
print(f"Average BERT Score: {sum(bert_scores) / len(bert_scores):.4f}")
print(f"Average Embedding Similarity: {sum(embedding_similarities) / len(embedding_similarities):.4f}")
print(f"Average Hybrid Score: {sum(hybrid_scores) / len(hybrid_scores):.4f}")

=== Evaluation Summary ===
Average ROUGE-L: 0.1457
Average BLEU: 2.25
Average F1: 0.1738
Average BERT Score: 0.8512
Average Embedding Similarity: 0.4932
Average Hybrid Score: 0.3591
